##### Sprintの目的
スクラッチを通してリカレントニューラルネットワークの基礎を理解する


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

from decimal import Decimal, ROUND_HALF_UP

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

#np.set_printoptions(threshold=0)

## リカレントニューラルネットワークスクラッチ

リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_t ・ W_x + h_{t-1}・W_h + B
$$
$$
h_t = tanh(a_t)
$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_{t}$ : 時刻tの入力 (batch_size, n_features)


$W_{x}$ : 入力に対する重み (n_features, n_nodes)


$h_{t-1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_{h}$ : 状態に対する重み。 (n_nodes, n_nodes)


$B$ : バイアス項 (n_nodes,)


初期状態 $h_{0}$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [2]:
# Initializer ############################
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=0.01):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W : ndarray(n_nodes1, n_nodes2)
        """
        W = self.sigma*np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B : ndarray(1, n_nodes2)
        """
        B = self.sigma*np.random.randn(1, n_nodes2)
        
        return B

# 活性化関数 ##########################
class Tanh:
    def forward(self, X):
        return np.tanh(X)

class Softmax:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def _softmax(self, X):
        X = X - np.max(X, axis=-1, keepdims=True)
        y = np.exp(X) / np.sum(np.exp(X), axis=-1, keepdims=True)
        
        return y
    
    def forward(self, X, t):
        self.t = t
        self.y = self._softmax(X)
        self.loss = self._cross_entropy_error(self.y, self.t)
        return self.loss
    
    def _cross_entropy_error(self, y, t):     
        batch_size = y.shape[0]
        return -np.sum(t * np.log(y + 1e-7)) / batch_size
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dX = (self.y - self.t)/batch_size
        return dX

# Optimizer #########################
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.W = layer.W - self.lr*layer.dW
        layer.B = layer.B - self.lr*layer.dB

In [3]:
class SimpleRNN:
    def __init__(self, n_nodes, n_features, initializer=None, optimizer=SGD(), activation=Tanh()):
        self.n_nodes = n_nodes
        self.n_features = n_features
        if initializer is None:
            self.Wx = w_x  # shape(n_features, n_nodes)
            self.Wh = w_h  # shape(n_nodes, n_nodes)
            self.B = b.astype(np.float)
        else:
            self.Wx = initializer.W(n_features, n_nodes)
            self.Wh = initializer.W(n_nodes, n_nodes)
            self.B = initializer.B(n_features, n_nodes)
        self.optimizer = optimizer
        self.activation = activation
        
    def forward(self, X):
        self.batch_size = X.shape[0]
        self.n_sequences = X.shape[1]
        self.h = np.zeros((self.batch_size, self.n_nodes)) # (batch_size, n_nodes)
        
        for i in range(self.batch_size):
            for j in range(self.n_sequences):
                A = (X[i][j] @ self.Wx) + (self.h @ self.Wh) + self.B
                self.h = self.activation.forward(A)
        return self.h

### 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [4]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

In [5]:
x

array([[[0.01, 0.02],
        [0.02, 0.03],
        [0.03, 0.04]]])

In [6]:
w_x

array([[0.01, 0.03, 0.05, 0.07],
       [0.03, 0.05, 0.07, 0.08]])

In [7]:
w_h

array([[0.01, 0.03, 0.05, 0.07],
       [0.02, 0.04, 0.06, 0.08],
       [0.03, 0.05, 0.07, 0.08],
       [0.04, 0.06, 0.08, 0.1 ]])

In [8]:
h

array([[0., 0., 0., 0.]])

**フォワードプロパゲーションの流れ確認**

In [9]:
#1

In [10]:
#x@wx
x[0][0]@w_x

array([0.0007, 0.0013, 0.0019, 0.0023])

In [11]:
#x@wx + b
x[0][0]@w_x + b

array([1.0007, 1.0013, 1.0019, 1.0023])

In [12]:
#tanh(x@wx + b) = h_s1
np.tanh(x[0][0]@w_x + b)

array([0.76188798, 0.76213958, 0.76239095, 0.76255841])

In [13]:
#h_s1@w_h
np.tanh(x[0][0]@w_x + b)@w_h

array([0.07623574, 0.13721527, 0.19819481, 0.25155044])

In [14]:
#2

In [15]:
#x@wx + (h_s1@w_h) + b
x[0][1]@w_x + np.tanh(x[0][0]@w_x + b)@w_h + b

array([1.07733574, 1.13931527, 1.20129481, 1.25535044])

In [16]:
#tanh(x@wx + (h_s1@w_h) + b) = h_s2
np.tanh(x[0][1]@w_x + np.tanh(x[0][0]@w_x + b)@w_h + b)

array([0.792209  , 0.8141834 , 0.83404912, 0.84977719])

In [17]:
#h_s2@w_h
(np.tanh(x[0][1]@w_x + np.tanh(x[0][0]@w_x + b)@w_h + b))@w_h

array([0.08321832, 0.14902269, 0.21482707, 0.27229095])

In [18]:
#3

In [19]:
#x@wx + (h_s2@w_h) + b
x[0][2]@w_x + (np.tanh(x[0][1]@w_x + np.tanh(x[0][0]@w_x + b)@w_h + b))@w_h + b

array([1.08471832, 1.15192269, 1.21912707, 1.27759095])

In [20]:
#tanh(x@wx + (h_s2@w_h) + b)
np.tanh(x[0][2]@w_x + (np.tanh(x[0][1]@w_x + np.tanh(x[0][0]@w_x + b)@w_h + b))@w_h + b)

array([0.79494228, 0.81839002, 0.83939649, 0.85584174])

#### 回答

In [21]:
rnn = SimpleRNN(n_nodes, n_features)
rnn.forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

**diverの例と一致していることを確認**

### 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。